In [2]:
from lib.ekyn import *
from sage.utils import *
from sage.models import *

sequence_length = 3
id = 'A1-1'
condition = 'Vehicle'

X,y = load_ekyn_pt(id=id,condition=condition)
X.shape,y.shape

(torch.Size([8640, 5000]), torch.Size([8640, 3]))

In [6]:
# Assuming X.shape is (num_samples, num_features) and y.shape is (num_samples, num_classes)
num_features = X.shape[1]
num_classes = y.shape[1]

In [7]:
# Pad the sequence
X = torch.cat([torch.zeros(sequence_length // 2, num_features), X, torch.zeros(sequence_length // 2, num_features)]).unsqueeze(1)
y = torch.cat([torch.zeros(sequence_length // 2, num_classes), y, torch.zeros(sequence_length // 2, num_classes)])

In [8]:
X.shape,y.shape

(torch.Size([8642, 1, 5000]), torch.Size([8642, 3]))

In [ ]:
from lib.ekyn import *
from sage.utils import *
from sage.models import *

class SequencedDatasetv2(torch.utils.data.Dataset):
    def __init__(self, id, condition, sequence_length, stride=1):
        self.sequence_length = sequence_length
        # X, y = load_ekyn_pt_robust(id=id, condition=condition, downsampled=True)
        X,y = load_ekyn_pt(id=id,condition=condition)
        
        # Assuming X.shape is (num_samples, num_features) and y.shape is (num_samples, num_classes)
        num_features = X.shape[1]
        num_classes = y.shape[1]
        
        # Pad the sequence
        self.X = torch.cat([torch.zeros(sequence_length // 2, num_features), X, torch.zeros(sequence_length // 2, num_features)]).unsqueeze(1)
        self.y = torch.cat([torch.zeros(sequence_length // 2, num_classes), y, torch.zeros(sequence_length // 2, num_classes)])
        
        self.stride = stride
        self.sequences = []
        self.labels = []
        for i in range(0, len(self.y) - sequence_length, stride):
            self.sequences.append(self.X[i:i + sequence_length])
            self.labels.append(self.y[i + sequence_length // 2])
        
        self.sequences = torch.stack(self.sequences)
        self.labels = torch.stack(self.labels)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]
    
def get_sequenced_dataloaders(batch_size=512,shuffle_train=True,shuffle_test=False,sequence_length=3,stride=1):
    from sklearn.model_selection import train_test_split
    ekyn_ids = get_ekyn_ids()

    train_ids,test_ids = train_test_split(ekyn_ids,test_size=.2,shuffle=True,random_state=0)

    from torch.utils.data import DataLoader,ConcatDataset
    trainloader = DataLoader(
            dataset=ConcatDataset(
            [SequencedDatasetv2(id=id,condition=condition,sequence_length=sequence_length,stride=stride) for id in train_ids for condition in ['Vehicle','PF']] 
            ),
            batch_size=batch_size,
            shuffle=shuffle_train,
            num_workers=1
        )
    testloader = DataLoader(
            dataset=ConcatDataset(
            [SequencedDatasetv2(id=id,condition=condition,sequence_length=sequence_length,stride=1) for id in test_ids for condition in ['Vehicle','PF']] 
            ),
            batch_size=batch_size,
            shuffle=shuffle_test,
            num_workers=1
        )
    return trainloader,testloader

trainloader,testloader = get_sequenced_dataloaders(batch_size=512,sequence_length=3,stride=1)